In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.metrics import silhouette_score

In [ ]:
# 1. Carica il DataFrame
user = pd.read_csv(
    '../datasets/ml-100k/u.user',
    sep='|',
    names=['user_id', 'age', 'gender', 'occupation', 'zip_code']
)

In [ ]:
# 2. Prepara “sentences” per Word2Vec includendo anche zip_code
sentences = user[['gender', 'occupation', 'zip_code']].astype(str).values.tolist()

# 3. Allena il modello Word2Vec
w2v = Word2Vec(
    sentences,
    vector_size=8,    # dimensione del vettore di embedding
    window=2,
    min_count=1,
    epochs=100,
    seed=42
)

In [ ]:
# 4. Definisci funzione per calcolare embedding medio per riga
def embed_row(row):
    vecs = [w2v.wv[row['gender']], 
            w2v.wv[row['occupation']], 
            w2v.wv[row['zip_code']]]
    # media vettoriale
    return sum(vecs) / len(vecs)


In [ ]:
# 5. Applica embedding a tutto il DataFrame
embeddings = user.apply(embed_row, axis=1)
df_emb = pd.DataFrame(
    embeddings.tolist(),
    columns=[f'emb_{i}' for i in range(w2v.vector_size)]
)
# Aggiungiamo eventualmente altre feature numeriche (es. age)
df_emb['age'] = user['age']

In [ ]:
sil_scores = {}
range_n_clusters = range(2, 30)
for k in range_n_clusters:
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(df_emb)
    sil = silhouette_score(df_emb, labels)
    sil_scores[k] = sil
    print(f"k = {k} → silhouette score = {sil:.4f}")

best_k = max(sil_scores, key=sil_scores.get)
print(f"\nNumero ottimale di cluster: k = {best_k} (silhouette = {sil_scores[best_k]:.4f})")

In [ ]:
# 6. Clustering sui vettori di embedding
kmeans_final = KMeans(n_clusters=4, random_state=42)
user['cluster_w2v'] = kmeans_final.fit_predict(df_emb)

In [ ]:
# 7. Riduzione a 3D per visualizzazione
pca = PCA(n_components=3, random_state=42)
coords3d = pca.fit_transform(df_emb)
df_plot = pd.DataFrame(coords3d, columns=['PC1', 'PC2', 'PC3'])
df_plot['cluster'] = user['cluster_w2v'].astype(str)

In [ ]:
# 8. Grafico 3D interattivo con Plotly
fig = px.scatter_3d(
    df_plot,
    x='PC1', y='PC2', z='PC3',
    color='cluster',
    title='Word2Vec Embedding + KMeans: visualizzazione 3D',
    labels={'cluster':'Cluster'}
)
fig.update_traces(marker=dict(size=4))
fig.show()